In [1]:
import pyspark 
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf, SparkContext
import os
import re
import numpy as np
import time
from functools import reduce
import pandas as pd

Location_master = 'spark://linux:7077'
Location_master = 'local'
try:
    sparkSession.stop()
except:
    pass
sparkSession = SparkSession \
    .builder \
    .master(Location_master)\
    .appName("Process 2020") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.executor.memory", "4g") \
    .config('spark.cores.max','4')\
    .getOrCreate()

In [20]:
data_dir = '../../../Data_process/*/*'
data_all = sparkSession.read.csv(data_dir, header=True)

data_all = data_all.drop('_c0', 'KHTN', 'KHXH', 'Cum_thi')
data_all = data_all.withColumn('Cum_thi', f.substring(f.col('SBD'), 0, 2))
data_all.show(5)
data_all.printSchema()
print(data_all.count())

+--------+----+-------+---------+------+-------+--------+-------+------+----+-------+
|     SBD|Toan|Ngu_van|Ngoai_ngu|Vat_ly|Hoa_hoc|Sinh_hoc|Lich_su|Dia_ly|GDCD|Cum_thi|
+--------+----+-------+---------+------+-------+--------+-------+------+----+-------+
|02000096| 7.8|   5.75|     null|  7.75|   6.75|     5.5|   null|  null|null|     02|
|02000095|   8|   null|     null|  null|   null|    null|   null|  null|null|     02|
|02000094|   8|   6.75|      8.8|   5.5|   6.25|    7.25|   null|  null|null|     02|
|02000092| 8.8|   6.75|        9|  8.75|   7.75|    3.25|   null|  null|null|     02|
|02000091|   8|   5.75|      9.2|  8.75|   8.25|     5.5|   null|  null|null|     02|
+--------+----+-------+---------+------+-------+--------+-------+------+----+-------+
only showing top 5 rows

root
 |-- SBD: string (nullable = true)
 |-- Toan: string (nullable = true)
 |-- Ngu_van: string (nullable = true)
 |-- Ngoai_ngu: string (nullable = true)
 |-- Vat_ly: string (nullable = true)
 |-- Ho

### Thay đổi kiểu dữ liệu các trường dữ liệu về đúng dạng của nó

In [21]:
import pyspark.sql.functions as F
import pyspark.sql.functions as f
from pyspark.sql.types import FloatType, StringType, IntegerType

In [22]:
data_all = data_all.drop('_c0')
data_all = data_all.withColumn('Toan', F.col('Toan').cast(FloatType()))
data_all = data_all.withColumn('Ngu_van', F.col('Ngu_van').cast(FloatType()))
data_all = data_all.withColumn('Ngoai_ngu', F.col('Ngoai_ngu').cast(FloatType()))
data_all = data_all.withColumn('Vat_ly', F.col('Vat_ly').cast(FloatType()))
data_all = data_all.withColumn('Hoa_hoc', F.col('Hoa_hoc').cast(FloatType()))
data_all = data_all.withColumn('Sinh_hoc', F.col('Sinh_hoc').cast(FloatType()))
data_all = data_all.withColumn('Lich_su', F.col('Lich_su').cast(FloatType()))
data_all = data_all.withColumn('Dia_ly', F.col('Dia_ly').cast(FloatType()))
data_all = data_all.withColumn('GDCD', F.col('GDCD').cast(FloatType()))
data_all = data_all.withColumn('SBD', F.col('SBD').cast(StringType()))
data_all = data_all.withColumn('Cum_thi', F.col('Cum_thi').cast(StringType()))

data_all.printSchema()

root
 |-- SBD: string (nullable = true)
 |-- Toan: float (nullable = true)
 |-- Ngu_van: float (nullable = true)
 |-- Ngoai_ngu: float (nullable = true)
 |-- Vat_ly: float (nullable = true)
 |-- Hoa_hoc: float (nullable = true)
 |-- Sinh_hoc: float (nullable = true)
 |-- Lich_su: float (nullable = true)
 |-- Dia_ly: float (nullable = true)
 |-- GDCD: float (nullable = true)
 |-- Cum_thi: string (nullable = true)



In [23]:
data_check = data_all.withColumn('crawl', f.substring(f.col('SBD'), 3, 4)).select('Cum_thi', 'crawl')
data_check = data_check.groupBy('Cum_thi', 'crawl').count().orderBy('Cum_thi', 'crawl')

In [24]:
data_all = data_all.drop_duplicates()
print(data_all.count())

988013


In [25]:
from pyspark.sql.window import Window

data_all = data_all.withColumn('row_num', f.row_number().over(
    Window.partitionBy('SBD').orderBy('SBD')
))

In [26]:
data_all.filter('row_num == 2').show()

+---+----+-------+---------+------+-------+--------+-------+------+----+-------+-------+
|SBD|Toan|Ngu_van|Ngoai_ngu|Vat_ly|Hoa_hoc|Sinh_hoc|Lich_su|Dia_ly|GDCD|Cum_thi|row_num|
+---+----+-------+---------+------+-------+--------+-------+------+----+-------+-------+
+---+----+-------+---------+------+-------+--------+-------+------+----+-------+-------+



In [8]:
# if(data_bosung.count() == 0):
#     data_dup = data_all.filter('row_num = 2').withColumn('raw', f.substring(f.col('SBD'),0,5))
#     data_dup = data_dup.groupBy('raw').count().toPandas()
#     list_data_miss = []
#     list_dup = list(data_dup['raw'].values)
#     for dup in list_dup:
#         for i in range(0,10):
#             pattern = dup + str(i)
#             if(data_all.filter("SBD like '{0}%'".format(pattern)).count() == 0):
#                 list_data_miss.append(pattern)

#     list_data_miss = pd.DataFrame({'Miss': list_data_miss})
#     list_data_miss.to_csv("../../../Data_process/ListMiss/miss.csv")

In [27]:
print(data_all.count())
data_all = data_all.filter('row_num == 1')
data_all = data_all.drop("row_num")
data_all = data_all.drop_duplicates()
print(data_all.count())
data_all = data_all.orderBy("SBD")
data_all.repartition(1).write.format("com.databricks.spark.csv")\
        .option("header", "true").save("mydata.csv", mode='overwrite')

988013
988013


In [28]:
sparkSession.stop()